In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!pip install transformers
!pip install -U -q PyDrive
!pip install 
!pip install keras
!pip install sentencepiece
!pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.8 MB/s eta 0:00:00
ERROR: You must give at least one requirement to install (see "pip help install")
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 18.2 MB/s eta 0:00:00


In [ ]:
!pip install pytorch-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 57.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 106.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.7 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=3bd335f39c6e03805ffe1fcb18351d2e96bb2373df71a81ad8762d9749f2cdb3
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


###Imports

In [ ]:
# Import required libraries
import torch
import pandas as pd
import numpy as np
import re
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score
from transformers import XLNetTokenizer, XLNetForMultipleChoice
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
import torch

# Confirm that the GPU is detected

assert torch.cuda.is_available()

# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")

Found device: Tesla T4, n_gpu: 1


### Data download and preprocessing

In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

def clean_text(text):
    # Convert all text to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(text)
    tokens = [token for token in tokens if token not in stop_words]
    text = ' '.join(tokens)

    # Remove extra whitespaces
    text = re.sub(' +', ' ', text)

    return text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:

import pandas as pd
df = pd.read_csv('/content/gdrive/MyDrive/NLP Project/data/IMDB Dataset.csv')
df = df.rename(columns={'review':'text'})
df = df[['text', 'sentiment']]

 
# How much of the dataset to use
data_size = 0.2
df = df.sample(frac=data_size, random_state=42)

df['text'] = df['text'].apply(clean_text)

# Convert the sentiment labels into numerical values
sentiment_map = {'positive': 0, 'negative': 1}
df['sentiment'] = df['sentiment'].replace(sentiment_map)

# Find and delete any empty rows
empty_rows = df[df['text'].apply(lambda x: isinstance(x, str) and len(x.strip()) == 0)]
df.drop(empty_rows.index, inplace=True)

In [ ]:
df.head()


,text,sentiment
33553,really liked summerslam due look arena curtain...,0
9427,many television shows appeal quite many differ...,0
199,film quickly gets major chase scene ever incre...,1
12447,jane austen would definitely approve onebr br ...,0
39489,expectations somewhat high went see movie thou...,1


In [ ]:
texts = df.text.values
texts = [text + " [SEP] [CLS]" for text in texts]
sentiment = df.sentiment.values

In [ ]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(text) for text in texts]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

Tokenize the first sentence:
['▁really', '▁liked', '▁summer', 's', 'lam', '▁due', '▁look', '▁arena', '▁curtains', '▁look', '▁overall', '▁interesting', '▁reason', '▁anyway', 's', '▁could', '▁one', '▁best', '▁summer', 's', 'lam', 's', '▁ever', '▁', 'ww', 'f', '▁didn', 't', '▁', 'lex', '▁', 'lug', 'er', '▁main', '▁event', '▁', 'y', 'oko', 'z', 'una', '▁time', '▁', 'ok', '▁huge', '▁fat', '▁man', '▁vs', '▁strong', '▁man', '▁im', '▁glad', '▁times', '▁changed', '▁terrible', '▁main', '▁event', '▁like', '▁every', '▁match', '▁', 'lug', 'er', '▁terrible', '▁matches', '▁card', '▁razor', '▁', 'ram', 'on', '▁vs', '▁', 'ted', '▁di', 'bia', 'se', '▁', 'stein', 'er', '▁brothers', '▁vs', '▁heavenly', '▁bodies', '▁', 'shaw', 'n', '▁mi', 'cha', 'els', '▁vs', '▁', 'cur', 't', '▁he', 'ning', '▁event', '▁', 'shaw', 'n', '▁named', '▁big', '▁monster', '▁body', '▁guard', '▁diesel', '▁', 'ir', 's', '▁vs', '▁kid', '▁', 'bre', 't', '▁', 'hart', '▁first', '▁takes', '▁do', 'ink', '▁takes', '▁', 'jer', 'ry', '▁law', 

In [ ]:
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [ ]:
# Pad our input tokens
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAX_LEN = 512

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [ ]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

### Train/val/test splits

In [ ]:
from sklearn.model_selection import train_test_split




# Splits train/test/val -> 70/20/10
train_inputs, temp_inputs, train_labels, temp_labels = train_test_split(input_ids, sentiment,
                                                                        random_state=42, test_size=0.3)

validation_inputs, test_inputs, validation_labels, test_labels = train_test_split(temp_inputs, temp_labels,
                                                                        random_state=42, test_size=0.33)

# Masks in 70/20/10
train_masks, temp_masks, _, _ = train_test_split(attention_masks, input_ids,
                                                  random_state=42, test_size=0.3)
# 
validation_masks, test_masks, _, _ = train_test_split(temp_masks, temp_inputs,
                                                  random_state=42, test_size=0.33)

# Splitting train set into train and validation (70% train, 20% validation)
# train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(train_inputs, train_labels,
#                                                                                     random_state=42, test_size=0.2)

# train_masks, validation_masks, _, _ = train_test_split(train_masks, train_inputs,
#                                                        random_state=42, test_size=0.2)



In [ ]:
# Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

test_inputs = torch.tensor(test_inputs)
test_masks = torch.tensor(test_masks)
test_labels = torch.tensor(test_labels)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# Select a batch size for training. For fine-tuning with XLNet, the authors recommend a batch size of 32, 48, or 128. We will use 32 here to avoid memory issues.
batch_size = 16

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


### Training

In [ ]:
import torch
from transformers import XLNetForSequenceClassification, XLNetTokenizer

model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0-11): 12 x XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (sequence_summary): SequenceSummary(
    (summary): Linear(in_features=768, out_features=768, bias=True)
    (activation): Tanh()
    (first_dropout): Identity()
    (last

In [ ]:

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]


In [ ]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters,
                     lr=2e-5)

total_steps = len(train_data) * 4
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
'''# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)'''

'# Function to calculate the accuracy of our predictions vs labels\ndef flat_accuracy(preds, labels):\n    pred_flat = np.argmax(preds, axis=1).flatten()\n    labels_flat = labels.flatten()\n    return np.sum(pred_flat == labels_flat) / len(labels_flat)'

In [ ]:
'''#Commented for testing
from tqdm import tqdm, trange
best_val_loss = float("inf")
num_epochs = 2
train_loss = []
val_loss = []
for epoch in range(num_epochs):
  # Train the model
  model.train()
  epoch_loss = 0

  train_loop = tqdm(train_dataloader, desc=f'Training Epoch {epoch+1}', leave=True)

  for batch in train_loop:
      
    inputs = {'input_ids': batch[0].to(device),
              'attention_mask': batch[1].to(device),
              'labels': batch[2].to(device)}
    optimizer.zero_grad()
    outputs = model(**inputs)
    loss = outputs[0]

    loss.backward()
    optimizer.step()
    scheduler.step()

    epoch_loss += loss.item()
    train_loop.set_postfix(loss=loss.item())

  epoch_loss /= len(train_dataloader)
  train_loss.append(epoch_loss)

  # Evaluate the model on the validation set
  model.eval()
  val_preds = []
  val_labels = []
  epoch_val_loss = 0

  with torch.no_grad():
    for batch in validation_dataloader:
        
      inputs = {'input_ids': batch[0].to(device),
                'attention_mask': batch[1].to(device),
                'labels': batch[2].to(device)}
      
      outputs = model(**inputs)
      loss = outputs[0]
      epoch_val_loss += loss.item()

      logits = outputs[1]
      preds = torch.argmax(logits, axis=1)
      val_preds.extend(preds.cpu().numpy())
      val_labels.extend(batch[2].cpu().numpy())

    epoch_val_loss /= len(validation_dataloader)

  if epoch_val_loss < best_val_loss:
    best_val_loss = epoch_val_loss
    
    path = '/content/gdrive/MyDrive/NLP Project/models/XLNet_SA_clean100'

    torch.save(model.state_dict(), path+'/model_parameters.pth')

  # Compute the evaluation metrics
  val_accuracy = accuracy_score(val_labels, val_preds)
  val_report = classification_report(val_labels, val_preds, target_names=['positive', 'negative'])
  

  # Print the results for the current epoch
  print('Epoch:', epoch+1, ', Training Loss:', epoch_loss/len(train_dataloader), ', Validation Loss:', epoch_val_loss, ', Validation Accuracy:', val_accuracy)
  print('Validation Classification Report:')
  print(val_report)'''

'#Commented for testing\nfrom tqdm import tqdm, trange\nbest_val_loss = float("inf")\nnum_epochs = 2\ntrain_loss = []\nval_loss = []\nfor epoch in range(num_epochs):\n  # Train the model\n  model.train()\n  epoch_loss = 0\n\n  train_loop = tqdm(train_dataloader, desc=f\'Training Epoch {epoch+1}\', leave=True)\n\n  for batch in train_loop:\n      \n    inputs = {\'input_ids\': batch[0].to(device),\n              \'attention_mask\': batch[1].to(device),\n              \'labels\': batch[2].to(device)}\n    optimizer.zero_grad()\n    outputs = model(**inputs)\n    loss = outputs[0]\n\n    loss.backward()\n    optimizer.step()\n    scheduler.step()\n\n    epoch_loss += loss.item()\n    train_loop.set_postfix(loss=loss.item())\n\n  epoch_loss /= len(train_dataloader)\n  train_loss.append(epoch_loss)\n\n  # Evaluate the model on the validation set\n  model.eval()\n  val_preds = []\n  val_labels = []\n  epoch_val_loss = 0\n\n  with torch.no_grad():\n    for batch in validation_dataloader:\n   

###Evaluation on clean data

In [ ]:
#Load model for evaluations, comment for finetuning

path = '/content/gdrive/MyDrive/NLP Project/models/XLNet_SA_clean100'
model.load_state_dict(torch.load(path+'/model_parameters.pth'))

<All keys matched successfully>

In [ ]:
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
model.eval()  # Set model to evaluation mode

predicted_labels = []
true_labels = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Set the device

model.to(device)  # Move the model to the same device as the input tensors

for batch in test_dataloader:
    inputs, masks, labels = batch

    inputs = inputs.to(device)
    masks = masks.to(device)
    labels = labels.to(device)

    with torch.no_grad():  # Disable gradient tracking
        outputs = model(inputs, attention_mask=masks)

    predicted_labels.extend(torch.argmax(outputs.logits, dim=1).cpu().tolist())  # Get predicted labels
    true_labels.extend(labels.cpu().tolist())  # Get true labels

accuracy = accuracy_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels, average='weighted')

print("Accuracy on clean data:", accuracy)
print("F1 Score on clean data:", f1)


Accuracy on clean data: 0.9111111111111111
F1 Score on clean data: 0.9110882517289675


###Evaluation on noisy data

In [ ]:
# Noise funcs 

char_action = ['insert',
        'substitute',
        'delete',
        'swap',
]

word_action = ['substitute',
        'delete',
        'swap',
]


def get_action(type):
  if type=="char":
    return random.choice(char_action)
  elif type=="word":
    return random.choice(word_action)


def augment_tweet(tweet, p=0.7):
    """
    Augment a tweet with character-level and word-level noise.

    Args:
        tweet (str): The original tweet.
        p (float): The probability of applying the char level augmentation.

    Returns:
        str: The augmented tweet.
    """
    # Define a list of character-level augmentation techniques
    char_augmenters = [
        nac.OcrAug(),
        nac.KeyboardAug(aug_char_p=0.2, aug_word_p=0.2, include_special_char=False),
        nac.RandomCharAug(action=get_action("char"), aug_char_p=0.1, aug_word_p=0.1),
    ]

    # Define a list of word-level augmentation techniques
    word_augmenters = [
        naw.SpellingAug(),
        naw.SplitAug(),
        naw.SynonymAug(),
        naw.RandomWordAug(aug_p=0.2, action=get_action("word")),
    ]

    # Randomly apply a character-level or word-level augmentation with probability p
    if random.random() < p:
        aug = random.choice(char_augmenters)
        augmented_tweet = aug.augment(tweet)
    else:
        aug = random.choice(word_augmenters)
        augmented_tweet = aug.augment(tweet)
        
    return augmented_tweet

def add_noise(df, augmentation_percentage, task):

  if task=="sentiment_analysis":
    # Sample 10% of the rows in the DataFrame
    augment_indices = df.sample(frac=augmentation_percentage).index

    # Apply the augment_tweet function to each tweet in the sampled rows
    for index in augment_indices:
        tweet = df.loc[index, 'text']
        augmented_tweet = augment_tweet(tweet)
        df.loc[index, 'text'] = augmented_tweet
    
    return df
  
  elif task=="question_answering":

    #noise functions for QA

    return df

In [ ]:
#load df again for testing on a random percent of noisy data
import random
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
test_df_noisy = pd.read_csv('/content/gdrive/MyDrive/NLP Project/data/IMDB Dataset.csv')
test_df_noisy = test_df_noisy.rename(columns={'review':'text'})
test_df_noisy = test_df_noisy[['text', 'sentiment']]

#add random noise
sentiment_map = {'positive': 0, 'negative': 1}
test_df_noisy['sentiment'] = test_df_noisy['sentiment'].replace(sentiment_map)

# Find and delete any empty rows
empty_rows = test_df_noisy[test_df_noisy['text'].apply(lambda x: isinstance(x, str) and len(x.strip()) == 0)]
test_df_noisy.drop(empty_rows.index, inplace=True)

random.seed(42)
random_noise = random.uniform(0.05, 0.15)
print(random_noise)

test_df_noisy = add_noise(test_df_noisy, augmentation_percentage=random_noise, task="sentiment_analysis")


# aadd SEP and CLS tokens

texts = test_df_noisy.text.values
texts = [''.join(text) + " [SEP] [CLS]" for text in texts]
sentiment = test_df_noisy.sentiment.values


#Tokenize

tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(text) for text in texts]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

MAX_LEN = 512

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")




0.11394267984578837


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Tokenize the first sentence:
['▁one', '▁of', '▁the', '▁other', '▁reviewer', 's', '▁has', '▁mentioned', '▁that', '▁after', '▁watching', '▁just', '▁1', '▁', 'oz', '▁episode', '▁you', "'", 'll', '▁be', '▁hooked', '.', '▁they', '▁are', '▁right', ',', '▁as', '▁this', '▁is', '▁exactly', '▁what', '▁happened', '▁with', '▁me', '.', '<', 'br', '▁', '/', '>', '<', 'br', '▁', '/', '>', 'the', '▁first', '▁thing', '▁that', '▁struck', '▁me', '▁about', '▁', 'oz', '▁was', '▁its', '▁brutality', '▁and', '▁un', 'fli', 'nch', 'ing', '▁scenes', '▁of', '▁violence', ',', '▁which', '▁set', '▁in', '▁right', '▁from', '▁the', '▁word', '▁go', '.', '▁trust', '▁me', ',', '▁this', '▁is', '▁not', '▁a', '▁show', '▁for', '▁the', '▁faint', '▁', 'hearted', '▁or', '▁timid', '.', '▁this', '▁show', '▁pulls', '▁no', '▁punches', '▁with', '▁regards', '▁to', '▁drugs', ',', '▁sex', '▁or', '▁violence', '.', '▁its', '▁is', '▁hardcore', ',', '▁in', '▁the', '▁classic', '▁use', '▁of', '▁the', '▁word', '.', '<', 'br', '▁', '/', '>', '<

In [ ]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)


# Splits train/test/val -> 70/20/10
train_inputs, temp_inputs, train_labels, temp_labels = train_test_split(input_ids, sentiment,
                                                                        random_state=42, test_size=0.3)

validation_inputs, test_inputs, validation_labels, test_labels = train_test_split(temp_inputs, temp_labels,
                                                                        random_state=42, test_size=0.33)

# Masks in 70/20/10
train_masks, temp_masks, _, _ = train_test_split(attention_masks, input_ids,
                                                  random_state=42, test_size=0.3)
# 
validation_masks, test_masks, _, _ = train_test_split(temp_masks, temp_inputs,
                                                  random_state=42, test_size=0.33)


test_inputs = torch.tensor(test_inputs)
test_masks = torch.tensor(test_masks)


test_labels = torch.tensor(test_labels)

In [ ]:
path = '/content/gdrive/MyDrive/NLP Project/models/XLNet_SA_clean100'
model.load_state_dict(torch.load(path+'/model_parameters.pth'))
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
model.eval()  # Set model to evaluation mode

predicted_labels = []
true_labels = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Set the device

model.to(device)  # Move the model to the same device as the input tensors

for batch in test_dataloader:
    inputs, masks, labels = batch

    inputs = inputs.to(device)
    masks = masks.to(device)
    labels = labels.to(device)

    with torch.no_grad():  # Disable gradient tracking
        outputs = model(inputs, attention_mask=masks)

    predicted_labels.extend(torch.argmax(outputs.logits, dim=1).cpu().tolist())  # Get predicted labels
    true_labels.extend(labels.cpu().tolist())  # Get true labels

accuracy = accuracy_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels, average='weighted')

print("Accuracy on x% random noisy data:", accuracy)
print("F1 Score on x% random noisy data:", f1)


Accuracy on x% random noisy data: 0.9385858585858586
F1 Score on x% random noisy data: 0.9385598619976434
